# Cifar10 Challenge
ref)
- https://github.com/adhishthite/cifar10-optimizers
- https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33

## Import Library

In [14]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input

from keras import layers
from keras import optimizers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras_applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs

import os
import numpy as np
import warnings


## Load Data

In [2]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = False
num_predictions = 20

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Categorical

In [3]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Network (ResNet50)
![ResNet](https://t1.daumcdn.net/cfile/tistory/99167C335C47F0E315)

![Skip Connection1](https://miro.medium.com/max/1458/1*uyXEvYztiv3fGGCCPbm8Jg.png)
![Skip Connection2](https://miro.medium.com/max/1294/1*U5wkA4O1IpY-ekXqFh0tUQ.png)

In [11]:
def conv131_block(input_tensor, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
        
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # 1x1
    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    # 3x3
    x = Conv2D(filters2, (3, 3), padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    # 1x1
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    # skip connection 
    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    
    return x

def identity131_block(input_tensor, filters, stage, block):
    filters1, filters2, filters3 = filters

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # 1x1
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    # 3x3
    x = Conv2D(filters2, (3, 3), padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    # 1x1
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    # skip connection
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    
    return x

In [23]:
cifar10_classes = 10
img_width, img_height = 32, 32
image_input = Input(shape=(img_width, img_height, 3))

# stage 1
# 7x7, 64, stride 2
x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(image_input)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, name='bn_conv1')(x)
x = Activation('relu')(x)

# stage 2: conv2_x
# 3x3 max pool, stride 2
x = MaxPooling2D((3, 3), strides=(2, 2))(x)
x = conv131_block(x, [64, 64, 256], strides=(2, 2), stage=2, block='a')
x = identity131_block(x, [64, 64, 256], stage=2, block='b')
x = identity131_block(x, [64, 64, 256], stage=2, block='c')

# stage 3: conv3_x
x = conv131_block(x, [128, 128, 512], strides=(2, 2), stage=3, block='a')
x = identity131_block(x, [128, 128, 512], stage=3, block='b')
x = identity131_block(x, [128, 128, 512], stage=3, block='c')
x = identity131_block(x, [128, 128, 512], stage=3, block='d')

# stage 4: conv4_x
x = conv131_block(x, [256, 256, 1024], strides=(2, 2), stage=4, block='a')
x = identity131_block(x, [256, 256, 1024], stage=4, block='b')
x = identity131_block(x, [256, 256, 1024], stage=4, block='c')
x = identity131_block(x, [256, 256, 1024], stage=4, block='d')
x = identity131_block(x, [256, 256, 1024], stage=4, block='e')
x = identity131_block(x, [256, 256, 1024], stage=4, block='f')

# stage 5: conv5_x
x = conv131_block(x, [512, 512, 2048], stage=5, block='a')
x = identity131_block(x, [512, 512, 2048], stage=5, block='b')
x = identity131_block(x, [512, 512, 2048], stage=5, block='c')

# flatten and softmax
x = Flatten(name='flattenx')(x)
x = Dense(cifar10_classes, activation='softmax', name='fc10')(x)

# Create model.
model = Model(image_input, x, name='resnet50')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [24]:
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [25]:
batch_size = 32
epochs = 20

his_tranin = model.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 133s 3ms/step - loss: 2.5073 - acc: 0.1944 - val_loss: 2.1963 - val_acc: 0.2660
Epoch 2/20
50000/50000 [==============================] - 123s 2ms/step - loss: 1.7301 - acc: 0.3757 - val_loss: 1.6378 - val_acc: 0.4074
Epoch 3/20
50000/50000 [==============================] - 123s 2ms/step - loss: 1.4610 - acc: 0.4816 - val_loss: 1.4231 - val_acc: 0.4947
Epoch 4/20
50000/50000 [==============================] - 123s 2ms/step - loss: 1.3122 - acc: 0.5394 - val_loss: 1.3192 - val_acc: 0.5259
Epoch 5/20
50000/50000 [==============================] - 123s 2ms/step - loss: 1.1993 - acc: 0.5844 - val_loss: 1.3918 - val_acc: 0.5263
Epoch 6/20
50000/50000 [==============================] - 123s 2ms/step - loss: 1.1204 - acc: 0.6135 - val_loss: 1.3493 - val_acc: 0.5357
Epoch 7/20
50000/50000 [==============================] - 125s 3ms/step - loss: 1.0299 - acc: 0.6462 - val_loss: 1.5460 - 

In [27]:
epochs_range = range(1, 20)
validation_accuracy = his_tranin.history['val_acc']
training_accuracy = his_tranin.history['acc']

TypeError: 'History' object is not subscriptable

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(epochs_range, training_accuracy, 'b+', label='training accuracy')
# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs_range, validation_accuracy, 'bo', label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Validation accuracy')
plt.legend()

plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])